In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

from skimage.color import rgb2gray, gray2rgb

Using TensorFlow backend.


In [2]:
def load_data():
        color = []
        bw = []
        paths = []
        #get all files in this folder
        for r, d, f in os.walk(r"E:\DF\DFL2\workspace\bw"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))
        #for each file add normal resolution and small resolution to arrays
        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            bw.append(np.array(x))
        del paths    
        paths = []
        for r, d, f in os.walk(r"E:\DF\DFL2\workspace\color"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            color.append(np.array(x))
            
        #reshaping data to be four dimension required for input to neural network
        y_train = np.array(color)
        y_train = y_train.reshape(len(color),128,128,3)
        x_train = np.array(bw)
        x_train = x_train.reshape(len(bw),128,128,3)
        return y_train, x_train

In [6]:
class SRGAN():
    def __init__(self):
        
        # Shape of high resolution output image
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Shape of low resolution input image
        self.latent_dim = (128,128,3)

        #optimizer (learning rate and beta values)
        optimizer = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=self.latent_dim)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Conv2D(140, input_shape=(128, 128, 3), kernel_size=(3,3), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(4,4), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=self.latent_dim)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        Y_train, X_train = load_data()

        # Rescale to be between 0 & 1
        X_train = X_train / 255
        Y_train = Y_train / 255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Placeholder for loss function values
        g_loss_epochs = np.zeros((epochs, 1))
        d_loss_epochs = np.zeros((epochs, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, Y_train.shape[0], batch_size)
            imgs = Y_train[idx]

            # Generate super resolution images from the random batch of images
            gen_imgs = self.generator.predict(X_train[idx])

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(X_train[idx], valid)
            
            #save loss history
            g_loss_epochs[epoch] = g_loss
            d_loss_epochs[epoch] = d_loss[0]

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, X_train, idx)
                
        return g_loss_epochs, d_loss_epochs

    def save_imgs(self, epoch, X_train, idx):
        r, c = 3, 3
        # Select 9 random images
        index = np.random.randint(0, X_train.shape[0], 9)
        images = X_train[idx]
        # Super resolution the images
        gen_imgs = self.generator.predict(images)
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)
        # Plot each image
        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        # save model to .h5 file
        self.generator.save("generator.h5")

In [7]:
gan = SRGAN()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)       

In [8]:
g_loss, d_loss = gan.train(epochs=15001, batch_size=9, save_interval=100)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow

0 [D loss: 1.241861, acc.: 38.89%] [G loss: 0.643162]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.698089, acc.: 55.56%] [G loss: 1.060630]
2 [D loss: 0.983266, acc.: 27.78%] [G loss: 1.045596]
3 [D loss: 0.782481, acc.: 44.44%] [G loss: 1.431036]
4 [D loss: 0.797676, acc.: 44.44%] [G loss: 1.488469]
5 [D loss: 1.516940, acc.: 16.67%] [G loss: 0.989677]
6 [D loss: 0.861779, acc.: 33.33%] [G loss: 0.991150]
7 [D loss: 0.918415, acc.: 55.56%] [G loss: 0.671598]
8 [D loss: 1.094389, acc.: 27.78%] [G loss: 0.870130]
9 [D loss: 1.107681, acc.: 33.33%] [G loss: 2.210557]
10 [D loss: 0.767895, acc.: 61.11%] [G loss: 1.550735]
11 [D loss: 0.568156, acc.: 66.67%] [G loss: 1.082131]
12 [D loss: 0.820658, acc.: 50.00%] [G loss: 1.641689]
13 [D loss: 1.011848, acc.: 33.33%] [G loss: 1.151701]
14 [D loss: 0.735364, acc.: 50.00%] [G loss: 1.045194]
15 [D loss: 0.565102, acc.: 66.67%] [G loss: 1.200214]
16 [D loss: 0.600385, acc.: 55.56%] [G loss: 1.198238]
17 [D loss: 0.634257, acc.: 72.22%] [G loss: 1.160763]
18 [D loss: 0.778912, acc.: 50.00%] [G loss: 0.946984]
19 [D loss: 0.63123

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


5300 [D loss: 0.054900, acc.: 100.00%] [G loss: 5.657694]
5301 [D loss: 0.030116, acc.: 100.00%] [G loss: 6.644346]
5302 [D loss: 0.099079, acc.: 94.44%] [G loss: 6.162218]
5303 [D loss: 0.070692, acc.: 94.44%] [G loss: 6.882111]
5304 [D loss: 0.021563, acc.: 100.00%] [G loss: 6.026284]
5305 [D loss: 0.040566, acc.: 100.00%] [G loss: 4.764351]
5306 [D loss: 0.013444, acc.: 100.00%] [G loss: 5.595057]
5307 [D loss: 0.023687, acc.: 100.00%] [G loss: 6.047376]
5308 [D loss: 0.058092, acc.: 100.00%] [G loss: 6.989233]
5309 [D loss: 0.031672, acc.: 100.00%] [G loss: 6.944943]
5310 [D loss: 0.084794, acc.: 100.00%] [G loss: 6.835189]
5311 [D loss: 0.042522, acc.: 100.00%] [G loss: 7.789089]
5312 [D loss: 0.022562, acc.: 100.00%] [G loss: 8.666778]
5313 [D loss: 0.031221, acc.: 100.00%] [G loss: 5.684662]
5314 [D loss: 0.092024, acc.: 94.44%] [G loss: 6.834345]
5315 [D loss: 0.018403, acc.: 100.00%] [G loss: 9.424722]
5316 [D loss: 0.006066, acc.: 100.00%] [G loss: 6.799337]
5317 [D loss: 0.0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


5400 [D loss: 0.059272, acc.: 100.00%] [G loss: 4.781893]
5401 [D loss: 0.017785, acc.: 100.00%] [G loss: 5.095572]
5402 [D loss: 0.039421, acc.: 100.00%] [G loss: 4.329530]
5403 [D loss: 0.035208, acc.: 100.00%] [G loss: 4.726703]
5404 [D loss: 0.311091, acc.: 77.78%] [G loss: 7.662988]
5405 [D loss: 0.035803, acc.: 100.00%] [G loss: 11.875012]
5406 [D loss: 0.398195, acc.: 88.89%] [G loss: 4.833623]
5407 [D loss: 0.031912, acc.: 100.00%] [G loss: 6.750617]
5408 [D loss: 0.342634, acc.: 83.33%] [G loss: 6.991236]
5409 [D loss: 0.022425, acc.: 100.00%] [G loss: 6.822062]
5410 [D loss: 0.106469, acc.: 94.44%] [G loss: 4.938700]
5411 [D loss: 0.003033, acc.: 100.00%] [G loss: 9.383029]
5412 [D loss: 0.152472, acc.: 94.44%] [G loss: 5.667368]
5413 [D loss: 0.008193, acc.: 100.00%] [G loss: 5.691996]
5414 [D loss: 0.031829, acc.: 100.00%] [G loss: 5.501588]
5415 [D loss: 0.528266, acc.: 77.78%] [G loss: 8.700382]
5416 [D loss: 0.126640, acc.: 94.44%] [G loss: 8.863685]
5417 [D loss: 0.1185

KeyboardInterrupt: 

In [ ]:
plt.plot(g_loss)
plt.plot(d_loss)
plt.title('GAN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Generator', 'Discriminator'], loc='upper left')
plt.show()